In [118]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [119]:
pd.options.display.max_columns = 100

In [120]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [121]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [122]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [123]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [124]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [125]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                                Titre          Titre alternatif  \
0    3539                              Yannick                       NaN   
1    3538                        Wild Sex Gang  狂走セックス族 (Titre original)   
2    3537                  Une année difficile                       NaN   
3    3536                Under the Cherry Moon                       NaN   
4    3535  This Film Is Not Yet Rated (VOSTFR)                       NaN   

  Année  Durée Chemin de l'affiche                        Bande-annonce  \
0  2023     67          003623.jpg  www.youtube.com/watch?v=wq85VKrV7C4   
1  1973     83          003622.jpg                                  NaN   
2  2023    120          003621.jpg  www.youtube.com/watch?v=lcOwBLN6YDk   
3  1986     98          003620.jpg  www.youtube.com/watch?v=IwkqXsWPY1k   
4  2006     98          003619.jpg  www.youtube.com/watch?v=UTL3XMDwY0c   

                                Synopsis  \
0                                    NaN   
1   

In [126]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [127]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [128]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [129]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [130]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [131]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [132]:
display(df_director)

,id,Réalisateur(s)
0,1,Quentin Dupieux
1,2,Takayuki Minagawa
2,3,Éric Toledano
3,4,Olivier Nakache
4,5,Prince
...,...,...
1752,1753,Richard C. Sarafian
1753,1754,Chris Petit
1754,1755,Brett Ratner
1755,1756,Buster Keaton


In [133]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,003623.jpg,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,003622.jpg,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,003621.jpg,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,003620.jpg,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,003619.jpg,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [134]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3539
1,0,3074
2,0,2810
3,0,1805
4,0,1758
...,...,...
3810,1752,20
3811,1753,18
3812,1754,15
3813,1755,5


In [135]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id     Réalisateur(s)
0  1758    Quentin Dupieux
1     2  Takayuki Minagawa
2     3      Éric Toledano
3     4    Olivier Nakache
4     5             Prince
     id  Numéro
0  1758    3539
1  1758    3074
2  1758    2810
3  1758    1805
4  1758    1758


3815

In [136]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [137]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [138]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3539

# table movie, screenwriter et intermediaire

In [139]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [140]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [141]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [142]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [143]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [144]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [145]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Quentin Dupieux
1,1,Norifumi Suzuki
2,2,Takayuki Minagawa
3,3,Éric Toledano
4,4,Olivier Nakache
...,...,...
3828,3828,Ted Tally
3829,3829,David Veloz
3830,3830,Richard Rutowski
3831,3831,Buster Keaton


In [146]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [147]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3539
1,0,3074
2,0,2810
3,0,1805
4,0,1758
...,...,...
6866,3828,15
6867,3829,14
6868,3830,14
6869,3831,5


In [148]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id      Scénariste(s)
0  3833    Quentin Dupieux
1     1    Norifumi Suzuki
2     2  Takayuki Minagawa
3     3      Éric Toledano
4     4    Olivier Nakache
     id  Numéro
0  3833    3539
1  3833    3074
2  3833    2810
3  3833    1805
4  3833    1758


6871

In [149]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [150]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [151]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [152]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [153]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [154]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [155]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [156]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [157]:
display(df_music)

,id,Compositeur(s)
0,0,NaN
1,1,Prince
2,2,Chumei Watanabe
3,3,Paul Simon
4,4,Ichirô Araki
...,...,...
1258,1258,Machine Head
1259,1259,Photek
1260,1260,John Frizzell
1261,1261,Leonard Cohen


In [158]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [159]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3539
1,0,3538
2,0,3537
3,0,3535
4,0,3532
...,...,...
3920,1258,46
3921,1259,46
3922,1260,42
3923,1261,14


In [160]:
# # Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
# null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# # Si une ligne avec 'Compositeur(s)' NULL est trouvée
# if not null_music_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_music.loc[null_music_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_music['id'].max()
#     if max_id > 0:
#         df_music.loc[df_music.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_music.head())

# # Enregistrer le DataFrame modifié dans la table 'music' de la base de données
# df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_music.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


In [161]:
df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

1263

In [162]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

3925

# table movie, studio et intermediaire

In [163]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [164]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [165]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [166]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [167]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [168]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [169]:
display(df_studio)

,id,Studio
0,0,CNC
1,1,Toei Company
2,2,Gaumont
3,3,Warner Bros. Pictures
4,4,BBC
...,...,...
929,929,A-Film Distribution
930,930,TFC
931,931,Dybex
932,932,British Empire Films Australia


In [170]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [171]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3539
1,0,3491
2,0,2751
3,0,2636
4,0,2304
...,...,...
3536,929,172
3537,930,171
3538,931,169
3539,932,108


In [172]:
# # Trouver l'index de la ligne avec le 'Studio' NULL
# null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# # Si une ligne avec 'Studio' NULL est trouvée
# if not null_studio_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_studio.loc[null_studio_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_studio['id'].max()
#     if max_id > 0:
#         df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_studio.head())

# # Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
# df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_studio.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


In [173]:
df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

934

In [174]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3541

# table movie, casting & intermediaire

In [175]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [176]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [177]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [178]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [179]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [180]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [181]:
display(df_casting)

,id,Distribution
0,0,Raphaël Quenard
1,1,Pio Marmaï
2,2,Blanche Gardin
3,3,Sébastien Chassagne
4,4,Agnès Hurstel
...,...,...
9723,9723,Paulette Goddard
9724,9724,Henry Bergman
9725,9725,Tiny Sandford
9726,9726,Jackie Coogan


In [182]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [183]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3539
1,1,3539
2,1,3537
3,1,3208
4,1,2516
...,...,...
16780,9723,2
16781,9724,2
16782,9725,2
16783,9726,1


In [184]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


     id         Distribution
0  9728      Raphaël Quenard
1     1           Pio Marmaï
2     2       Blanche Gardin
3     3  Sébastien Chassagne
4     4        Agnès Hurstel
     id  Numéro
0  9728    3539
1     1    3539
2     1    3537
3     1    3208
4     1    2516


16785

In [185]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [186]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [187]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [188]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [189]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [190]:
df_country_movie  = df_country_movie.explode("Pays")

In [191]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [192]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [193]:
display(df_country)

,id,Pays
0,0,France
1,1,Japon
2,2,Etats-Unis
3,3,Italie
4,4,Royaume-Uni
5,5,Suède
6,6,Afrique du Sud
7,7,Hong Kong
8,8,Allemagne
9,9,Corée du Sud


In [194]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [195]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3539
1,0,3537
2,0,3532
3,0,3530
4,0,3527
...,...,...
3534,42,853
3535,43,921
3536,44,769
3537,45,631


In [196]:
# # Trouver l'index de la ligne avec le 'Pays' NULL
# null_country_index = df_country[df_country['Pays'].isnull()].index

# # Si une ligne avec 'Pays' NULL est trouvée
# if not null_country_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_country.loc[null_country_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_country['id'].max()
#     if max_id > 0:
#         df_country.loc[df_country.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_country.head())

# # Enregistrer le DataFrame modifié dans la table 'country' de la base de données
# df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_country.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

In [197]:
df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

46

In [198]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3539

# table movie, genre & intermediaire

In [199]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [200]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [201]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [202]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [203]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [204]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [205]:
display(df_genre)

,id,Genre(s)
0,0,Comédie
1,1,Policier
2,2,Asia
3,3,Yakuzas
4,4,Romantique
5,5,Musical
6,6,Documentaire
7,7,Suspense
8,8,Drame
9,9,Science-fiction


In [206]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [207]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3539
1,0,3537
2,0,3533
3,0,3532
4,0,3530
...,...,...
7930,38,920
7931,38,642
7932,38,384
7933,38,352


In [208]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id    Genre(s)
0  39     Comédie
1   1    Policier
2   2        Asia
3   3     Yakuzas
4   4  Romantique
   id  Numéro
0  39    3539
1  39    3537
2  39    3533
3  39    3532
4  39    3530


7935

In [209]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [210]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

7935

# table movie, language & intermediaire

In [211]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [212]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [213]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [214]:
df_language_movie  = df_language_movie.explode("Langues")

In [215]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [216]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [217]:
display(df_language)

,id,Langues
0,0,Français
1,1,Japonais
2,2,Anglais
3,3,NaN
4,4,Allemand
5,5,Italien
6,6,Suédois
7,7,Cantonais
8,8,Turc
9,9,Russe


In [218]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [219]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3539
1,0,3537
2,0,3536
3,0,3532
4,0,3530
...,...,...
4978,42,1060
4979,43,1053
4980,43,930
4981,43,926


In [220]:
# # Trouver l'index de la ligne avec les 'Langues' NULL
# null_language_index = df_language[df_language['Langues'].isnull()].index

# # Si une ligne avec 'Langues' NULL est trouvée
# if not null_language_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_language.loc[null_language_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_language['id'].max()
#     if max_id > 0:
#         df_language.loc[df_language.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_language.head())

# # Enregistrer le DataFrame modifié dans la table 'language' de la base de données
# df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_language.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


In [221]:
df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

45

In [222]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

4983

# table, movie, tag & intermediaire

In [223]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [224]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [225]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [226]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [227]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [228]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [229]:
display(df_tag)

,id,Etiquettes
0,0,NaN
1,1,Voiture
2,2,Yakusa
3,3,Prison
4,4,Candide
...,...,...
201,201,Momie
202,202,viking
203,203,Réalisme soviétique
204,204,Guy Debord


In [230]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3539,Yannick,NaN,2023,67,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wq85VKrV7C4,NaN,En pleine représentation de la pièce « Le Cocu...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Yannick ...,Matroska (MKV),NaN,Fichier multimédia,3.30 GB,movie/1110358,28236368.0
1,3538,Wild Sex Gang,狂走セックス族 (Titre original),1973,83,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le riche punk Junya n'a qu'une idée en tête : ...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Wild Sex G...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/119144,222841.0
2,3537,Une année difficile,NaN,2023,120,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=lcOwBLN6YDk,NaN,Albert et Bruno sont surendettés et en bout de...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Une Anne...,Matroska (MKV),NaN,Fichier multimédia,2.55 GB,movie/965791,21352380.0
3,3536,Under the Cherry Moon,NaN,1986,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IwkqXsWPY1k,See It - Hear It - Feel It - Live It,"Originaires de Miami, deux frères, Tricky et C...",[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Under th...,Matroska (MKV),NaN,Fichier multimédia,1.21 GB,movie/33345,92133.0
4,3535,This Film Is Not Yet Rated (VOSTFR),NaN,2006,98,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=UTL3XMDwY0c,"Censorship, uncensored.",Le documentaire provocateur de Kirby Dick enqu...,[My Book]F:\#DIVX\DIVX MOVIEZ\DOC\This Film Is...,Matroska (MKV),NaN,Fichier multimédia,764 MB,movie/16070,493459.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3535,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3536,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3537,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [231]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3539
1,0,3537
2,0,3536
3,0,3535
4,0,3533
...,...,...
4486,204,74
4487,204,73
4488,205,17
4489,205,16


In [232]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id Etiquettes
0  206        NaN
1    1    Voiture
2    2     Yakusa
3    3     Prison
4    4    Candide
    id  Numéro
0  206    3539
1  206    3537
2  206    3536
3  206    3535
4  206    3533


4491

In [233]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [234]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )